# KKUI CIT Course - Neural networks - Week_05 - Multy layer perceptron

In [346]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.decomposition import PCA
import plotly.graph_objs as go
import plotly.io as pio
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix

## Support functions

In [347]:
def visualise_data(MY_X, MY_y):
    # Apply PCA to reduce data to 2 dimensions
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(MY_X)

    # Get target names
    target_names = MY_y

    # Create hover text for labels
    # hover_text = [f'Target: {target_names[label]}<br>Attributes (only first 5): {", ".join(map(str, attrs[0:5]))}'
    #               for label, attrs in zip(y, X)]

    # Create trace for data points
    trace = go.Scatter(
        x=X_pca[:, 0],
        y=X_pca[:, 1],
        mode='markers',
        # hovertext=hover_text,
        marker=dict(
            size=7,
            color=target_names,
            colorscale='Viridis',
            line=dict(
                color='rgb(0, 0, 0)',
                width=0.5
            ),
            opacity=0.8
        )
    )

    # Create layout
    layout = go.Layout(
        title='Data Distribution in 2D (PCA)',
        xaxis=dict(title='Principal Component 1'),
        yaxis=dict(title='Principal Component 2'),
        width=800,  # Set the width of the plot
        height=600,  # Set the height of the plot
    )

    # Create figure
    fig = go.Figure(data=[trace], layout=layout)

    # Show interactive plot
    pio.show(fig)

In [348]:
def plot_training_validation_metrics(train_accuracies, val_accuracies, train_losses, val_losses, model, optimizer, learning_rate):
    # Create subplots
    fig = make_subplots(rows=1, cols=2, subplot_titles=(
        f'Training and Validation Accuracy',
        f'Training and Validation Loss'
    ))

    # Plot training and validation accuracy
    fig.add_trace(go.Scatter(x=list(range(len(train_accuracies))), y=train_accuracies, mode='lines', name='Train Acc', line=dict(color='blue')), row=1, col=1)
    fig.add_trace(go.Scatter(x=list(range(len(val_accuracies))), y=val_accuracies, mode='lines', name='Val Acc', line=dict(color='orange')), row=1, col=1)

    # Plot training and validation loss
    fig.add_trace(go.Scatter(x=list(range(len(train_losses))), y=train_losses, mode='lines', name='Train Loss', line=dict(color='blue')), row=1, col=2)
    fig.add_trace(go.Scatter(x=list(range(len(val_losses))), y=val_losses, mode='lines', name='Val Loss', line=dict(color='orange')), row=1, col=2)

    # Update layout
    fig.update_layout(title_text=f"Training and Validation Metrics -> Model: {model.__class__.__name__}, Optimizer: {optimizer.__class__.__name__}, Learning Rate: {learning_rate}",
                      title_font=dict(size=18),
                      showlegend=True,
                      title_x=0.5)  # Center title

    # Update subplot titles
    fig.update_xaxes(title_text="Epoch", row=1, col=1)
    fig.update_yaxes(title_text="Accuracy", row=1, col=1)
    fig.update_xaxes(title_text="Epoch", row=1, col=2)
    fig.update_yaxes(title_text="Loss", row=1, col=2)

    # Show plot
    fig.show()

In [349]:



def calculate_metrics(model, X_test, y_test):
    with torch.no_grad():
        y_predicted = model(X_test)
        y_predicted_cls = (y_predicted > 0.5).float()
        acc = accuracy_score(y_test, y_predicted_cls, normalize=True)
        precision, recall, fscore, _ = precision_recall_fscore_support(y_test, y_predicted_cls, average='binary')
        cm = confusion_matrix(y_test, y_predicted_cls)
        print(f'Accuracy: {acc:.4f}')
        print(f'Precision: {precision:.4f}')
        print(f'Recall: {recall:.4f}')
        print(f'F-score: {fscore:.4f}')
        labels = ['Negative', 'Positive']  # Assuming binary classification
        fig = go.Figure(data=go.Heatmap(z=cm[::-1], x=labels, y=labels[::-1], colorscale='Blues', showscale=False))
        for i in range(len(labels)):
            for j in range(len(labels)):
                fig.add_annotation(x=labels[j], y=labels[::-1][i], text=str(cm[::-1][i][j]), showarrow=False, font=dict(color='black', size=14))
        fig.update_layout(title=f'Confusion Matrix -> Model: {model.__class__.__name__}', xaxis_title='Predicted Labels', yaxis_title='True Labels', width=500, height=500)
        fig.show()
        return acc, precision, recall, fscore, cm

## Prepare data

In [351]:
# create pandas dataframe
df = pd.read_csv("data/wine+quality/winequality-red.csv", delimiter=";")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [352]:
# Check dataset dimension
df.shape

(1599, 12)

In [356]:
set(df["quality"])

{3, 4, 5, 6, 7, 8}

In [354]:
df.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [357]:
# groupby("quality"): This method groups the DataFrame df by the values in the "quality" column. It creates a group for each unique value in the "quality" column. For example, if "quality" contains values like "Good", "Average", and "Poor", this method will create three groups: one for "Good" quality, one for "Average" quality, and one for "Poor" quality.
# count(): After grouping by the "quality" column, this method is applied to each group. It counts the number of occurrences of each value in the "quality" column within each group. Essentially, it counts the number of rows in each group.
df.groupby(["quality"]).count()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
quality,,,,,,,,,,,
3,10,10,10,10,10,10,10,10,10,10,10
4,53,53,53,53,53,53,53,53,53,53,53
5,681,681,681,681,681,681,681,681,681,681,681
6,638,638,638,638,638,638,638,638,638,638,638
7,199,199,199,199,199,199,199,199,199,199,199
8,18,18,18,18,18,18,18,18,18,18,18


In [358]:
# Convert quality range from integers to True False label. If quality marker is bigger then 5 mean that vine is qulite
df["label"] = 0
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,label
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,0
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,0
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,0
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,0


In [360]:
df.loc[df["quality"] >= 6, "label"] = 1

In [361]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,label
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,0
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,0


In [362]:
df.groupby("label").count()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
label,,,,,,,,,,,,
0,744,744,744,744,744,744,744,744,744,744,744,744
1,855,855,855,855,855,855,855,855,855,855,855,855


In [276]:
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality,label
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5,0
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5,0
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6,1
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5,0
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6,1
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6,1
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5,0


In [363]:
df.iloc[:,:-2]

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4
...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2


In [365]:
# Remove original "quality label" and our label 0 -> exclude last 2 columns from DF
X = df.iloc[:,:-2].values
y = df.iloc[:,-1].values

In [366]:
# Visualise data distribution via PCA
visualise_data(X,y)

### Dataset split and normalization

In [371]:
from sklearn.model_selection import train_test_split

# Splitting the dataset into training, testing, and validation sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=1234)


In [372]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_temp = sc.transform(X_temp)

In [373]:
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [374]:
# Convert numpy arrays to PyTorch tensors for training, validation, and testing data
X_train = torch.from_numpy(X_train.astype(np.float32))
X_val = torch.from_numpy(X_val.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_val = torch.from_numpy(y_val.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

# Reshape target tensors to have shape (batch_size, 1)
y_train = y_train.view(y_train.shape[0], 1)
y_val = y_val.view(y_val.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [378]:
X.shape

(1599, 11)

## Model prepair

In [396]:
# Linear model f = wx + b , sigmoid at the end
class MyModelLogistic(nn.Module):
    def __init__(self, n_input_features):
        super(MyModelLogistic, self).__init__()
        # Define a linear layer with input size n_input_features and output size 1
        self.linear = nn.Linear(n_input_features, 1)

    def forward(self, x):
        # Perform the linear transformation followed by the sigmoid activation function
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred


# Get the number of samples and features in the dataset
n_samples, n_features = X.shape

# Create an instance of the MyModelLogistic class with the number of input features as the argument
model = MyModelLogistic(n_features)

# Accessing parameters
for param in model.parameters():
    # Print each parameter of the model (weights and bias)
    print(param)



Parameter containing:
tensor([[-0.0058, -0.0703,  0.2231,  0.1675,  0.2081,  0.1310,  0.2595,  0.0929,
          0.0610,  0.2702, -0.2067]], requires_grad=True)
Parameter containing:
tensor([-0.2632], requires_grad=True)


In [397]:
from torchsummary import summary
summary(model, (1,n_features))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 1, 1]              12
Total params: 12
Trainable params: 12
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [398]:
num_epochs = 1000
learning_rate = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [399]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    # Forward pass and loss for training set
    model.train()  # Set the model to training mode
    train_outputs = model(X_train)
    train_loss = criterion(train_outputs, y_train)

    # Backward pass and update
    train_loss.backward()
    optimizer.step()

    # Zero gradients before new step
    optimizer.zero_grad()

    # Calculate training accuracy
    train_predictions = torch.round(train_outputs)
    train_correct = (train_predictions == y_train).sum().item()
    train_acc = train_correct / len(y_train)

    # Forward pass for validation set
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No need to compute gradients for validation
        val_outputs = model(X_val)
        val_loss = criterion(val_outputs, y_val)

        # Calculate validation accuracy
        val_predictions = torch.round(val_outputs)
        val_correct = (val_predictions == y_val).sum().item()
        val_acc = val_correct / len(y_val)

    # Store losses and accuracies
    train_losses.append(train_loss.item())
    val_losses.append(val_loss.item())
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    # Logging
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], '
              f'Train Loss: {train_loss.item():.4f}, Train Acc: {train_acc:.4f}, '
              f'Val Loss: {val_loss.item():.4f}, Val Acc: {val_acc:.4f}')


Epoch [10/1000], Train Loss: 0.8128, Train Acc: 0.4400, Val Loss: 0.8064, Val Acc: 0.4375
Epoch [20/1000], Train Loss: 0.8108, Train Acc: 0.4411, Val Loss: 0.8044, Val Acc: 0.4375
Epoch [30/1000], Train Loss: 0.8088, Train Acc: 0.4411, Val Loss: 0.8023, Val Acc: 0.4375
Epoch [40/1000], Train Loss: 0.8068, Train Acc: 0.4411, Val Loss: 0.8003, Val Acc: 0.4375
Epoch [50/1000], Train Loss: 0.8048, Train Acc: 0.4432, Val Loss: 0.7982, Val Acc: 0.4375
Epoch [60/1000], Train Loss: 0.8028, Train Acc: 0.4432, Val Loss: 0.7962, Val Acc: 0.4375
Epoch [70/1000], Train Loss: 0.8009, Train Acc: 0.4432, Val Loss: 0.7942, Val Acc: 0.4375
Epoch [80/1000], Train Loss: 0.7989, Train Acc: 0.4453, Val Loss: 0.7923, Val Acc: 0.4469
Epoch [90/1000], Train Loss: 0.7970, Train Acc: 0.4484, Val Loss: 0.7903, Val Acc: 0.4469
Epoch [100/1000], Train Loss: 0.7951, Train Acc: 0.4494, Val Loss: 0.7883, Val Acc: 0.4469
Epoch [110/1000], Train Loss: 0.7932, Train Acc: 0.4494, Val Loss: 0.7864, Val Acc: 0.4500
Epoch [1

In [400]:
plot_training_validation_metrics(train_accuracies, val_accuracies, train_losses, val_losses, model, optimizer, learning_rate)

### Model evaluation

In [401]:
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy: {acc.item():.4f}')

accuracy: 0.6281


### New evaluation metrics

In [403]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_predicted > 0.5, normalize=True)

0.628125

In [404]:
calculate_metrics(model, X_test, y_test)


Accuracy: 0.6281
Precision: 0.7054
Recall: 0.4788
F-score: 0.5704


(0.628125,
 0.7053571428571429,
 0.47878787878787876,
 0.5703971119133574,
 array([[122,  33],
        [ 86,  79]]))

In [250]:
from sklearn.metrics import precision_recall_fscore_support
precision_recall_fscore_support(y_test, y_predicted > 0.5, average="binary")

(0.768595041322314, 0.5636363636363636, 0.6503496503496503, None)

## Model - Multy layer perceptron

In [340]:
# Define the multilayer perceptron (MLP) model
class MLP(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, num_classes):
        super(MLP, self).__init__()

        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, hidden_size2)



        self.fc3 = nn.Linear(hidden_size2, 1)

        self.relu = nn.ReLU()
        self.sig = nn.Sigmoid()


    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)

        x = self.sig(self.fc3(x))
        return x

import torch.nn.functional as F

# class MLP(nn.Module):
#     def __init__(self, input_size, hidden_size1, hidden_size2, num_classes, dropout_prob=0.2):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(input_size, hidden_size1)
#         self.relu1 = nn.ReLU()
#         self.dropout1 = nn.Dropout(dropout_prob)
#         self.fc2 = nn.Linear(hidden_size1, hidden_size2)
#         self.relu2 = nn.ReLU()
#         self.dropout2 = nn.Dropout(dropout_prob)
#         self.fc3 = nn.Linear(hidden_size2, num_classes)
#
#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu1(x)
#         x = self.dropout1(x)
#         x = self.fc2(x)
#         x = self.relu2(x)
#         # x = self.dropout2(x)
#         x = torch.sigmoid(self.fc3(x))
#         return x

# # Define the multilayer perceptron (MLP) model
# class MLP(nn.Module):
#     def __init__(self, input_size, ratio):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(input_size, 32//ratio)
#         self.fc2 = nn.Linear(32//ratio, 128//ratio)
#         self.fc3 = nn.Linear(128//ratio, 64//ratio)
#         self.fc4 = nn.Linear(64//ratio, 32//ratio)
#         self.fc5 = nn.Linear(32//ratio, 1)
#         self.relu = nn.ReLU()
#         self.dropout1 = nn.Dropout(0.2)
#
#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu(x)
#         x = self.dropout1(x)
#         x = self.fc2(x)
#         x = self.relu(x)
#         x = self.fc3(x)
#         x = self.relu(x)
#         x = self.fc4(x)
#         x = self.relu(x)
#         x = torch.sigmoid(self.fc5(x))
#         return x

# # Define the multilayer perceptron (MLP) model
# class MLP(nn.Module):
#     def __init__(self, input_size, ratio):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(input_size, 32//ratio)
#         self.fc2 = nn.Linear(32//ratio, 128//ratio)
#         self.fc3 = nn.Linear(128//ratio, 64//ratio)
#         self.fc4 = nn.Linear(64//ratio, 32//ratio)
#         self.fc5 = nn.Linear(32//ratio, 1)
#         self.relu = nn.ReLU()
#         self.dropout1 = nn.Dropout(0.2)
#
#     def forward(self, x):
#         x = self.fc1(x)
#         x = self.relu(x)
#         # x = self.dropout1(x)
#         x = self.fc2(x)
#         x = self.relu(x)
#         x = self.fc3(x)
#         x = self.relu(x)
#         x = self.fc4(x)
#         x = self.relu(x)
#         x = torch.sigmoid(self.fc5(x))
#         return x


# Define model parameters

hidden_size1 = 32
hidden_size2 = 16
num_classes = 1

# hidden_size1 = 128
# hidden_size2 = 64
# num_classes = 1

# # Initialize the MLP model
# mlp_model = MLP(n_features, ratio=8)

# # Initialize the MLP model
mlp_model = MLP(n_features, hidden_size1, hidden_size2, num_classes)

num_epochs = 400
learning_rate = 0.001
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(mlp_model.parameters(), lr=learning_rate)

In [341]:
summary(mlp_model, (1,11))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                [-1, 1, 32]             384
              ReLU-2                [-1, 1, 32]               0
            Linear-3                [-1, 1, 16]             528
              ReLU-4                [-1, 1, 16]               0
            Linear-5                 [-1, 1, 1]              17
           Sigmoid-6                 [-1, 1, 1]               0
Total params: 929
Trainable params: 929
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00
----------------------------------------------------------------


In [342]:
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(num_epochs):
    # Forward pass and loss for training set
    model.train()  # Set the model to training mode
    train_outputs = mlp_model(X_train)
    train_loss = criterion(train_outputs, y_train)

    # Backward pass and update
    train_loss.backward()
    optimizer.step()

    # Zero gradients before new step
    optimizer.zero_grad()

    # Calculate training accuracy
    train_predictions = torch.round(train_outputs)
    train_correct = (train_predictions == y_train).sum().item()
    train_acc = train_correct / len(y_train)

    # Forward pass for validation set
    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():  # No need to compute gradients for validation
        val_outputs = mlp_model(X_val)
        val_loss = criterion(val_outputs, y_val)

        # Calculate validation accuracy
        val_predictions = torch.round(val_outputs)
        val_correct = (val_predictions == y_val).sum().item()
        val_acc = val_correct / len(y_val)

    # Store losses and accuracies
    train_losses.append(train_loss.item())
    val_losses.append(val_loss.item())
    train_accuracies.append(train_acc)
    val_accuracies.append(val_acc)

    # Logging
    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch + 1}/{num_epochs}], '
              f'Train Loss: {train_loss.item():.4f}, Train Acc: {train_acc:.4f}, '
              f'Val Loss: {val_loss.item():.4f}, Val Acc: {val_acc:.4f}')


Epoch [10/400], Train Loss: 0.6958, Train Acc: 0.4609, Val Loss: 0.6923, Val Acc: 0.4750
Epoch [20/400], Train Loss: 0.6830, Train Acc: 0.5881, Val Loss: 0.6806, Val Acc: 0.6312
Epoch [30/400], Train Loss: 0.6685, Train Acc: 0.6820, Val Loss: 0.6666, Val Acc: 0.6813
Epoch [40/400], Train Loss: 0.6504, Train Acc: 0.6955, Val Loss: 0.6486, Val Acc: 0.6656
Epoch [50/400], Train Loss: 0.6276, Train Acc: 0.7059, Val Loss: 0.6259, Val Acc: 0.6750
Epoch [60/400], Train Loss: 0.6006, Train Acc: 0.7059, Val Loss: 0.5992, Val Acc: 0.7000
Epoch [70/400], Train Loss: 0.5730, Train Acc: 0.7153, Val Loss: 0.5727, Val Acc: 0.7094
Epoch [80/400], Train Loss: 0.5493, Train Acc: 0.7268, Val Loss: 0.5501, Val Acc: 0.7250
Epoch [90/400], Train Loss: 0.5324, Train Acc: 0.7299, Val Loss: 0.5334, Val Acc: 0.7344
Epoch [100/400], Train Loss: 0.5213, Train Acc: 0.7351, Val Loss: 0.5228, Val Acc: 0.7375
Epoch [110/400], Train Loss: 0.5131, Train Acc: 0.7414, Val Loss: 0.5167, Val Acc: 0.7406
Epoch [120/400], Tr

In [343]:
epoch_of_val_loss_increase

399

In [344]:
plot_training_validation_metrics(train_accuracies, val_accuracies, train_losses, val_losses, mlp_model, optimizer, learning_rate)

In [345]:
calculate_metrics(mlp_model, X_test, y_test)


Accuracy: 0.7469
Precision: 0.7530
Recall: 0.7576
F-score: 0.7553


(0.746875,
 0.7530120481927711,
 0.7575757575757576,
 0.7552870090634441,
 array([[114,  41],
        [ 40, 125]]))